# Import necessary modules

In [2]:
import tensorflow as tf
import time

import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

# Load dataset

In [2]:
path_to_file = "text_generation.txt"

# Data pre-processing

In [5]:
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

59 unique characters


# Tokenization

In [7]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

# Vectorization
convert each character into a numeric ID

In [9]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

# Joining
tf.strings.reduce_join to join the characters back into strings.

In [12]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

 tf.data.Dataset.from_tensor_slices function to convert the text vector into a stream of character indices.


In [13]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(27270,), dtype=int64, numpy=array([24, 37, 49, ...,  1,  2,  1], dtype=int64)>

In [14]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [15]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

R
e
q
u
e
s
t
 
f
o


In [16]:
seq_length = 100

batch method converts these individual characters to sequences of the desired size.

In [17]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'R' b'e' b'q' b'u' b'e' b's' b't' b' ' b'f' b'o' b'r' b' ' b'M' b'a'
 b't' b'e' b'r' b'n' b'i' b't' b'y' b' ' b'L' b'e' b'a' b'v' b'e' b':'
 b'\r' b'\n' b'\r' b'\n' b'D' b'e' b'a' b'r' b' ' b'[' b'M' b'a' b'n' b'a'
 b'g' b'e' b'r' b"'" b's' b' ' b'N' b'a' b'm' b'e' b']' b',' b'\r' b'\n'
 b'\r' b'\n' b'I' b' ' b'a' b'm' b' ' b'w' b'r' b'i' b't' b'i' b'n' b'g'
 b' ' b't' b'o' b' ' b'i' b'n' b'f' b'o' b'r' b'm' b' ' b'y' b'o' b'u'
 b' ' b't' b'h' b'a' b't' b' ' b'I' b' ' b'w' b'i' b'l' b'l' b' ' b'b'
 b'e' b' ' b't'], shape=(101,), dtype=string)


In [18]:
sequences

<BatchDataset element_spec=TensorSpec(shape=(101,), dtype=tf.int64, name=None)>

In [19]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b"Request for Maternity Leave:\r\n\r\nDear [Manager's Name],\r\n\r\nI am writing to inform you that I will be t"
b'aking maternity leave starting from [Start Date] until [End Date]. As per the company policy, I am en'
b'titled to [Duration of Leave] weeks of maternity leave.\r\n\r\nI have completed all my pending work and h'
b'ave handed over my responsibilities to [Name of Colleague/Team Member]. I have also shared my contact'
b' details with them in case of any emergency.\r\n\r\nDuring my absence, I can be reached via [Email/Phone '


For training need a dataset of (input, label) pairs. 

Where input and label are sequences. 

At each time step the input is the current character and the label is the next character.

In [20]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [22]:
sequences

<BatchDataset element_spec=TensorSpec(shape=(101,), dtype=tf.int64, name=None)>

In [23]:
dataset = sequences.map(split_input_target)

dataset

<MapDataset element_spec=(TensorSpec(shape=(100,), dtype=tf.int64, name=None), TensorSpec(shape=(100,), dtype=tf.int64, name=None))>

In [24]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b"Request for Maternity Leave:\r\n\r\nDear [Manager's Name],\r\n\r\nI am writing to inform you that I will be "
Target: b"equest for Maternity Leave:\r\n\r\nDear [Manager's Name],\r\n\r\nI am writing to inform you that I will be t"


# Create training batches

In [25]:
# Batch size
BATCH_SIZE = 4

# Buffer size to shuffle the dataset
BUFFER_SIZE = 100

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(4, 100), dtype=tf.int64, name=None), TensorSpec(shape=(4, 100), dtype=tf.int64, name=None))>

# Model building

In [26]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [27]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [28]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [29]:
list(dataset)

[(<tf.Tensor: shape=(4, 100), dtype=int64, numpy=
  array([[52,  3, 35, 47, 46, 39, 50, 33, 52, 53, 44, 33, 52, 41, 47, 46,
          51,  3, 52, 47,  3, 57, 47, 53,  3, 47, 46,  3, 57, 47, 53, 50,
           3, 50, 37, 35, 37, 46, 52,  3, 33, 48, 48, 47, 41, 46, 52, 45,
          37, 46, 52,  3, 52, 47,  3, 52, 40, 37,  3, 31, 21, 37, 55,  3,
          24, 47, 44, 37, 32,  3, 33, 52,  3, 31, 12, 47, 45, 48, 33, 46,
          57,  3, 21, 33, 45, 37, 32,  7,  3, 17, 52,  3, 41, 51,  3, 52,
          50, 53, 44, 57],
         [33, 52, 37,  3, 57, 47, 53, 50,  3, 53, 46, 36, 37, 50, 51, 52,
          33, 46, 36, 41, 46, 39,  3, 33, 46, 36,  3, 44, 47, 47, 43,  3,
          38, 47, 50, 55, 33, 50, 36,  3, 52, 47,  3, 50, 37, 52, 53, 50,
          46, 41, 46, 39,  3, 52, 47,  3, 55, 47, 50, 43,  3, 33, 38, 52,
          37, 50,  3, 45, 57,  3, 44, 37, 33, 54, 37,  7,  2,  1,  2,  1,
          25, 41, 46, 35, 37, 50, 37, 44, 57,  5,  2,  1, 31, 30, 47, 53,
          50,  3, 21, 33],
        

In [30]:
list(dataset.take(1))

[(<tf.Tensor: shape=(4, 100), dtype=int64, numpy=
  array([[ 3, 33, 50, 37,  3, 33, 46, 57,  3, 33, 36, 36, 41, 52, 41, 47,
          46, 33, 44,  3, 51, 52, 37, 48, 51,  3, 17,  3, 46, 37, 37, 36,
           3, 52, 47,  3, 52, 33, 43, 37,  3, 52, 47,  3, 41, 46, 41, 52,
          41, 33, 52, 37,  3, 52, 40, 41, 51,  3, 48, 50, 47, 35, 37, 51,
          51,  7,  3, 17,  3, 44, 47, 47, 43,  3, 38, 47, 50, 55, 33, 50,
          36,  3, 52, 47,  3, 50, 37, 52, 53, 50, 46, 41, 46, 39,  3, 52,
          47,  3, 55, 47],
         [37, 33, 51, 37,  3, 44, 37, 52,  3, 45, 37,  3, 43, 46, 47, 55,
           3, 41, 38,  3, 52, 40, 37, 50, 37,  3, 33, 50, 37,  3, 33, 46,
          57,  3, 33, 36, 36, 41, 52, 41, 47, 46, 33, 44,  3, 51, 52, 37,
          48, 51,  3, 17,  3, 46, 37, 37, 36,  3, 52, 47,  3, 52, 33, 43,
          37,  3, 52, 47,  3, 41, 46, 41, 52, 41, 33, 52, 37,  3, 52, 40,
          41, 51,  3, 48, 50, 47, 35, 37, 51, 51,  7,  3, 17,  3, 33, 48,
          48, 50, 37, 35],
        

In [31]:
for input_example_batch, target_example_batch in dataset.take(1):
    print("building")
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "(batch_size, sequence_length, vocab_size)")

building
(4, 100, 60) (batch_size, sequence_length, vocab_size)


In [32]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  15360     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  61500     
                                                                 
Total params: 4,015,164
Trainable params: 4,015,164
Non-trainable params: 0
_________________________________________________________________


In [33]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [34]:
sampled_indices

array([40,  5, 34, 10, 29, 38, 51, 40, 56, 30, 10, 27, 27,  3, 21, 52, 42,
       17, 48,  8, 25, 59, 51, 11, 58, 49,  3, 51, 54, 55, 55, 41, 44, 37,
       37, 17, 27, 31, 49, 28,  5, 36, 25, 42, 26, 50, 36, 15,  7, 40, 25,
       16, 47, 24,  9, 48, 34, 32, 36, 24, 32,  2, 38, 26, 54,  0, 35, 21,
        6,  9, 45, 33, 16, 44, 29,  8, 10, 36, 20, 13, 18, 57, 31, 22, 18,
       53, 10, 14, 38, 36, 59, 42, 30, 57, 56, 13, 14, 42, 29, 41],
      dtype=int64)

In [35]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'eciate your understanding and look forward to returning to work after my leave.\r\n\r\nSincerely,\r\n[Your'

Next Char Predictions:
 b'h,bAWfshxYAUU NtjIp/S\xe2\x80\x99sBzq svwwileeIU[qV,dSjTrdF.hSHoR:pb]dR]\rfTv[UNK]cN-:maHlW/AdMDJy[OJuAEfd\xe2\x80\x99jYyxDEjWi'


# Train the model

In [36]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [37]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (4, 100, 60)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.0925136, shape=(), dtype=float32)


In [38]:
tf.exp(example_batch_mean_loss).numpy()

59.89024

In [39]:
model.compile(optimizer='adam', loss=loss)

In [40]:
EPOCHS = 100

# Training customized model

In [42]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [43]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [44]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [45]:
model.fit(dataset, epochs=100)

Epoch 1/100
67/67 [==============================] - 3s 18ms/step - loss: 3.0983
Epoch 2/100
67/67 [==============================] - 1s 15ms/step - loss: 2.0007
Epoch 3/100
67/67 [==============================] - 1s 15ms/step - loss: 1.5186
Epoch 4/100
67/67 [==============================] - 1s 15ms/step - loss: 1.1409
Epoch 5/100
67/67 [==============================] - 1s 15ms/step - loss: 0.8908
Epoch 6/100
67/67 [==============================] - 1s 15ms/step - loss: 0.7043
Epoch 7/100
67/67 [==============================] - 1s 15ms/step - loss: 0.5648
Epoch 8/100
67/67 [==============================] - 1s 15ms/step - loss: 0.4480
Epoch 9/100
67/67 [==============================] - 1s 15ms/step - loss: 0.3606
Epoch 10/100
67/67 [==============================] - 1s 15ms/step - loss: 0.2870
Epoch 11/100
67/67 [==============================] - 1s 15ms/step - loss: 0.2410
Epoch 12/100
67/67 [==============================] - 1s 15ms/step - loss: 0.2154
Epoch 13/100
67/67 [=====

# prediction function

In [46]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [47]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [48]:
start = time.time()
states = None
next_char = tf.constant(['setting the price'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

setting the pricest ry absence. I have completed all of my outstanding tasks and will ensure that my colleagues have  

________________________________________________________________________________

Run time: 0.7174360752105713


# Saving & Loading the model

In [49]:
tf.saved_model.save(one_step_model, 'one_step3')

INFO:tensorflow:Assets written to: one_step3\assets


INFO:tensorflow:Assets written to: one_step3\assets


In [3]:
one_step_reloaded = tf.saved_model.load('one_step3')

2023-03-10 15:11:52.861500: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-10 15:11:54.033117: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'while' has 11 outputs but the _output_shapes attribute specifies shapes for 21 outputs. Output shapes may be inaccurate.


# Running prediction model in loop to generate the text

In [4]:
states = None
next_char = tf.constant(['Leave:'])
result = [next_char]

for n in range(500):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

Leave:

Dear [Manager’s Name],

I am writing to request a sabbatical leave from work. I am interested in taking leave from [start date] to [end date]. I assure you that I will complete all pending work before my departure and will ensure a smooth handover to my colleague. Please let me know if this is possible.
Thank you for your consideration.

Sincerely,
[Your Name]


Request for Hospitalization Leave:

Dear [Manager’s Name],

I am writing to request a half-day leave of absence on [Da


In [5]:
for n in range(100):
    next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
    result.append(next_char)

op = tf.strings.join(result)[0].numpy().decode("utf-8")
# print("output",op)
# print(op.index)
try:
    print(op[:op.index("\n\r\n\r\n")])
    # print("try block")
except:
   print(op,"\n except block")
# #    print("except block")

Leave:

Dear [Manager’s Name],

I am writing to request a sabbatical leave from work. I am interested in taking leave from [start date] to [end date]. I assure you that I will complete all pending work before my departure and will ensure a smooth handover to my colleague. Please let me know if this is possible.
Thank you for your consideration.

Sincerely,
[Your Name]
